# Custom unit operations

New and emerging processing technologies often require custom tailored unit operations. In this problem set, we will leverage BioSTEAM's extendable framework to model fermentation for ethanol production. Mainly, we will:

1. Inherit design and costing algorithms from the abstract StirredTankReactor class and specify the mass and energy balance.
2. Test our results.

**Instructions:** Replace all `None`s with the correct values.

**Step 1: Inherit from the StirredTankReactor class**

* Two outlets: vent, broth

* Operating temperature and pressure: 32 degC and 1 atm.

* Reaction time: 60 hr.

* Ethanol yield: 90 % theoretical.

* Cell growth: 70 % of remaining glucose by weight.


In [1]:
import biosteam as bst
bst.nbtutorial()
class Fermentation(bst.StirredTankReactor):
    _N_ins = 1 # Number of inlets
    _N_outs = None # Number of outlets
    T_default = None # Operating temperature [K]
    P_default = None # Pressure [Pa]
    tau_default = None # Reaction time [hr]
    V_max_default = 3785 # Max volume per vessel [m3]
    V_wf_default = 0.95 # Working volume fraction
    auxiliary_unit_names = (
        'preheater', 
        *bst.StirredTankReactor.auxiliary_unit_names
    )

    def _run(self):
        fermentation = None # Define reaction with bst.Reaction
        cell_growth = bst.Reaction('Glucose -> Yeast', 'Glucose', 0.70, basis='wt')
        
        feed, = None # Get inlets 
        vent, effluent = None # Get outlets
        
        self.auxiliary(
            'preheater', bst.HXutility, 
            ins=None, # Feed from parent 
            T=None # Outlet temperature
        )
        self.preheater.simulate()
        self.parallel['preheater'] = 1
        heated_feed = self.preheater.outs[0]
        effluent.copy_like(heated_feed)
        
        # Isothermally react effluent
        fermentation(effluent)
        cell_growth(effluent)
        
        # Perform vapor-liquid equilibrium to find vent
        effluent.vle(T=self.T, P=self.P)
        vent.copy_like(effluent['g'])
        effluent['g'].empty()
        

**Step 2: Test results**

* Check diagram
* Check streams
* Check design and cost

In [2]:
from biorefineries.corn import create_chemicals
chemicals = create_chemicals()
bst.settings.set_thermo(None)
slurry = bst.Stream(
    'slurry', 
    Water=71.24, 
    Glucose=20.66, 
    TriOlein=1.153, 
    Fiber=3.646, 
    SolubleProtein=1.248,
    InsolubleProtein=1.523, 
    Ash=0.4254, 
    Yeast=0.01096, 
    CaO=0.003109, 
    H2SO4=0.0313,
    NH3=0.05181,
    total_flow=1.52e+05,
    units='kg/hr'
)
R1 = Fermentation(
    ins=None, 
    outs=('CO2', 'product'),
    batch=True,
)
R1.simulate()
R1.diagram()
R1.show('cwt100')
R1.results(include_installed_cost=True)

Fermentation: R1
ins...
[0] slurry
    phase: 'l', T: 298.15 K, P: 101325 Pa
    composition (%): Water             71.2
                     Glucose           20.7
                     Ash               0.425
                     Yeast             0.011
                     CaO               0.00311
                     TriOlein          1.15
                     H2SO4             0.0313
                     NH3               0.0518
                     Fiber             3.65
                     SolubleProtein    1.25
                     InsolubleProtein  1.52
                     ----------------  1.52e+05 kg/hr
outs...
[0] CO2
    phase: 'g', T: 305.15 K, P: 101325 Pa
    composition (%): Water    1.62
                     Ethanol  4.93
                     CO2      93.4
                     -------  6.05e+04 kg/hr
[1] product
    phases: ('g', 'l'), T: 305.15 K, P: 101325 Pa
    composition (%): (l) Water             60.9
                         Ethanol           30.4
          

Fermentation                                                    Units                   R1
Electricity              Power                                     kW             1.11e+04
                         Cost                                  USD/hr                  872
Low pressure steam       Duty                                   kJ/hr             3.83e+06
                         Flow                                 kmol/hr                 98.9
                         Cost                                  USD/hr                 23.5
Chilled water            Duty                                   kJ/hr            -6.39e+08
                         Flow                                 kmol/hr             4.28e+05
                         Cost                                  USD/hr              3.2e+03
Design                   Reactor volume                            m3              2.6e+03
                         Batch time                                hr                   84
                         Loading time                              hr                   21
                         Residence time                            hr                   60
                         Vessel type                                              Vertical
                         Length                                    ft                  102
                         Diameter                                  ft                 33.9
                         Weight                                    lb             3.64e+05
                         Wall thickness                            in                0.605
                         Vessel material                               Stainless steel 316
Purchase cost            Vertical pressure vessel (x4)            USD             4.74e+06
                         Platform and ladders (x4)                USD             5.84e+05
                         Preheater - Floating head (x4)           USD             2.07e+04
                         Heat exchanger - Floating head (x4)      USD             4.05e+06
                         Recirculation pump - Pump (x4)           USD              3.4e+06
                         Recirculation pump - Motor (x4)          USD             1.18e+05
                         Agitator - Agitator (x4)                 USD             1.66e+06
Total purchase cost                                               USD             1.46e+07
Installed equipment cost                                          USD             3.19e+07
Utility cost                                                   USD/hr             4.09e+03